In [1]:
from cognitive.task_bank import SequentialCategoryMatch
from cognitive import stim_generator as sg
from cognitive import task_generator as tg

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
"""
class SequentialCategoryMatch(TemporalTask):
    def __init__(self, whens=None, first_shareable=None, n_frames=1):
        super(SequentialCategoryMatch, self).__init__(whens=whens, first_shareable=first_shareable)
        total_frames = n_frames * 2 + random.randint(0, const.MAX_MEMORY - (n_frames * 2) + 1)

        sample_objs = [tg.Select(when=f'last{total_frames - i - 1}') for i in range(n_frames)]
        response_objs = [tg.Select(when=f'last{i}') for i in range(n_frames)]
        sample_cats = [tg.GetCategory(obj) for obj in sample_objs]
        response_cats = [tg.GetCategory(obj) for obj in response_objs]
        is_sames = [tg.IsSame(sample, response) for sample, response in zip(sample_cats, response_cats)]
        if n_frames == 1:
            self._operator = is_sames[0]
        else:
            ands = tg.And(is_sames[0], is_sames[1])
            for is_same1, is_same2 in zip(is_sames[2::2], is_sames[3::2]):
                ands = tg.And(tg.And(is_same1, is_same2), ands)
            self._operator = ands
        self.n_frames = total_frames

    @property
    def instance_size(self):
        return sg.n_sample_shape(2) * (sg.n_random_when()) ** 2

"""

# Initialize Parameters

In [ ]:
max_memory = 12
max_distractors=0
trials_per_family=1
output_dir='./data'
stim_dir='./data/MULTIF_5_stim'
random_families=True
families=['CompareLoc']
composition=3
temporal_switch=0
img_size=224
training =1.0
validation = 0
nback = 1
nback_length = 6
seq_length = 6
seq_reverse = False
fix_delay = False
fixation_cue = True


whens = [f'last{nback}', 'last0']
const.DATA = const.Data(args.stim_dir)
assert all('Compare' in family for family in args.families)
composition = args.nback_length - args.nback + 1

examples_per_family=trials_per_family
output_dir=output_dir
composition=composition
img_size=img_size
random_families=random_families
families=families
train=training
validation=validation
fixation_cue=fixation_cue
max_memory=max_memory
max_distractors=max_distractors
whens=whens
first_shareable=1
temporal_switch=temporal_switch


if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [2]:
CompareLocTemporal

def generate_temporal_example(, task_family,
                              whens=None, first_shareable=None, *args, **kwargs):
    # max_memory


    task = CompareLocTemporal(task_family, whens, first_shareable)
    assert isinstance(task, tg.TemporalTask)

    # To get maximum memory duration, we need to specify the following average
    # memory value
    avg_mem = round(args['max_memory'] / 3.0 + 0.01, 2)

    objset = task.generate_objset(n_distractor=0,average_memory_span=avg_mem, *args, **kwargs)
            # objset: sg.ObjectSet, epoch_now: int, should_be=None, temporal_switch=False

    # Getting targets can remove some objects from objset.
    # Create example fields after this call.
    frame_info = ig.FrameInfo(task, objset)
    compo_info = ig.TaskInfoCompo(task, frame_info)
    return compo_info

In [ ]:

def generate_temporal_example(max_memory, max_distractors, task_family,
                              whens=None, first_shareable=None, *args, **kwargs):
    """
    generate 1 task objset and composition info object

    :param max_memory:
    :param max_distractors:
    :param task_family:
    :param whens:
    :param first_shareable:
    :return:
    """
    task = task_bank.random_task(task_family, whens, first_shareable)
    assert isinstance(task, tg.TemporalTask)

    # To get maximum memory duration, we need to specify the following average
    # memory value
    avg_mem = round(max_memory / 3.0 + 0.01, 2)
    if max_distractors == 0:
        objset = task.generate_objset(average_memory_span=avg_mem, *args, **kwargs)
    else:
        objset = task.generate_objset(n_distractor=random.randint(1, max_distractors),
                                      average_memory_span=avg_mem, *args, **kwargs)
    # Getting targets can remove some objects from objset.
    # Create example fields after this call.
    frame_info = ig.FrameInfo(task, objset)
    compo_info = ig.TaskInfoCompo(task, frame_info)
    return compo_info


def generate_compo_temporal_example(max_memory, max_distractors, families, n_tasks=1,
                                    *args, **kwargs):
    '''

    :param first_shareable:
    :param whens:
    :param families:
    :param max_memory:
    :param max_distractors:
    :param n_tasks:
    :return: combined TaskInfo Compo
    '''

    whens = kwargs.pop('whens', [None for _ in range(n_tasks)])

    if not isinstance(whens[0], list):
        whens = [whens for _ in range(n_tasks)]
    if n_tasks == 1:
        return generate_temporal_example(max_memory, max_distractors, families, whens=whens[0], *args, **kwargs)

    compo_tasks = [generate_temporal_example(max_memory, max_distractors, families, whens=whens[i], *args, **kwargs)
                   for i in range(n_tasks)]
    # temporal combination
    cur_task = compo_tasks[0]
    for task in compo_tasks[1:]:
        cur_task.merge(task)
    return cur_task


# TODO: split training and validation after task generation

def generate_dataset(examples_per_family, output_dir='./data',
                     random_families=True, families=None,
                     composition=1, img_size=224,
                     train=0.7, validation=0.3, fixation_cue=True,
                     *args, **kwargs):
    if not random_families:
        assert families is not None
        assert composition == len(families)
    assert train + validation == 1.0
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # TODO: add task_family order instance folders
    #  e.g. families = [CompareLoc, CompareCat] could have 4 different task orders
    families_count = defaultdict(lambda: 0)
    if families is None or families == 'all':
        families = list(task_bank.task_family_dict.keys())
    print(families)
    n_families = len(families)
    total_examples = n_families * examples_per_family
    train_examples = total_examples * train
    validation_examples = total_examples * validation

    # creating task folder names
    fam_str = '_'.join(families)
    if train != 0.7 and validation != 0.3:
        base_fname = os.path.join(output_dir,
                                  f'tasks_{fam_str}_{train}_{validation}')
    else:
        base_fname = os.path.join(output_dir,
                                  f'tasks_{fam_str}')

    train_fname = os.path.join(base_fname, 'train')
    validation_fname = os.path.join(base_fname, 'validation')

    if not os.path.exists(train_fname):
        os.makedirs(train_fname)
    if not os.path.exists(validation_fname):
        os.makedirs(validation_fname)

    # TODO: temporal_switch generation pipeline. User specify compo vs switch or randomly.
    #  If randomly, how to split compo and switch?
    if random_families:
        p = np.random.permutation(total_examples)
        i = 0
        while i < len(p):
            if i % 10000 == 0 and i > 0:
                print("Generated ", i, " examples")

            task_family = list()
            for j in range(composition):
                task_family.append(families[p[i] % n_families])
                families_count[families[p[i] % n_families]] += 1

            info = generate_compo_temporal_example(families=task_family, n_tasks=composition, *args, **kwargs)
            # Write the example randomly to training or validation folder
            split = bool(random.getrandbits(1))
            if (split or validation_examples <= 0) and train_examples > 0:
                train_examples -= 1
                fname = os.path.join(train_fname, f'{i}')
            else:
                validation_examples -= 1
                fname = os.path.join(validation_fname, f'{i}')

            write_task_instance(fname, info, img_size, fixation_cue)
            i += 1
    else:
        i = 0
        while i < total_examples:
            if i % 10000 == 0 and i > 0:
                print("Generated ", i, " examples")
            task_family = np.random.permutation(families)

            compo_tasks = [generate_temporal_example(task_family=[family], *args, **kwargs)
                           for family in task_family]
            # temporal combination
            info = compo_tasks[0]
            for task in compo_tasks[1:]:
                info.merge(task)
            # TODO: find boolean output task and apply temporal switch
            #  draw diagram for branching structure of temporal composition/switch
            #  put in info_generator?
            info.temporal_switch()

            # Write the example randomly to training or validation folder
            split = bool(random.getrandbits(1))
            if (split or validation_examples <= 0) and train_examples > 0:
                train_examples -= 1
                fname = os.path.join(train_fname, f'{i}')
            else:
                validation_examples -= 1
                fname = os.path.join(validation_fname, f'{i}')

            write_task_instance(fname, info, img_size, fixation_cue)
            i += 1
    return families_count


def main():
    args = get_args()
    print(args)

    const.DATA = const.Data(args.stim_dir)

    start = timeit.default_timer()
    if args.nback > 0:
        assert all('Compare' in family for family in args.families)
        whens = [f'last{args.nback}', 'last0']
        composition = args.nback_length - args.nback + 1
        generate_dataset(examples_per_family=args.trials_per_family, output_dir=args.output_dir,
                         composition=composition, img_size=args.img_size,
                         random_families=args.random_families, families=args.families,
                         train=args.training, validation=args.validation, fixation_cue=args.fixation_cue,
                         max_memory=args.max_memory, max_distractors=args.max_distractors,
                         whens=whens, first_shareable=1, temporal_switch=args.temporal_switch)
    elif args.seq_length > 0:
        first_shareable = 1

        if args.seq_reverse:
            # minimum 2 frames per DMS task
            if args.seq_length * 2 - 1 > const.MAX_MEMORY:
                raise ValueError('Not enough max memory')
            whens = [[f'last{const.MAX_MEMORY - (i * 2)}', 'last0'] for i in range(args.seq_length)]
        else:
            last_when = sg.random_when()
            while int(re.search(r'\d+', last_when).group()) - 1 < args.seq_length:
                last_when = sg.random_when()
            whens = [[last_when, 'last0'] for _ in range(args.seq_length)]
        generate_dataset(examples_per_family=args.trials_per_family, output_dir=args.output_dir,
                         composition=args.seq_length, img_size=args.img_size,
                         random_families=args.random_families, families=args.families,
                         train=args.training, validation=args.validation, fixation_cue=args.fixation_cue,
                         max_memory=args.max_memory, max_distractors=args.max_distractors,
                         whens=whens, first_shareable=first_shareable, temporal_switch=args.temporal_switch)
    else:
        whens = ["last0", "last1"] # xlei: previously Mark set this as None
        if args.fix_delay:
            # TODO: move this into task.init
            whens = [f'last{const.DATA.MAX_MEMORY}', 'last0']
        generate_dataset(examples_per_family=args.trials_per_family, output_dir=args.output_dir,
                         composition=args.composition, img_size=args.img_size,
                         random_families=args.random_families, families=args.families,
                         train=args.training, validation=args.validation, fixation_cue=args.fixation_cue,
                         max_memory=args.max_memory, max_distractors=args.max_distractors,
                         whens=whens, first_shareable=None, temporal_switch=args.temporal_switch)